In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import plot_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a function to plot time series data
def plot_time_series(timesteps, values, format='.', start=0, end=None, label=None):
  """
  Plot a timesteps (a series of points in time) against values (a series values across timesteps).

  Args:
    timesteps: array of timesteps.
    values: array of values across timesteps.
    format: style of plot, default to ".".
    start: where to start the plot (setting a value for the index of timesteps & values start from)
    ens: where to end the plot (setting a value for the index of timesteps & values end at)
    label: label to show on plot of values
  """
  plt.plot(timesteps[start:end], values[start:end], format, label=label)
  plt.xlabel('Date')
  plt.ylabel('power')
  if label:
    plt.legend(fontsize=14)
  plt.grid(True)

In [ ]:
def mean_absolute_scaled_error(y_true, y_pred):
  """
  Implement MASE (assuming no seasonality).
  """
  mae = tf.reduce_mean(tf.abs(y_true - y_pred))

  mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1]))

  MASE = mae/mae_naive_no_season

  return MASE

In [ ]:
def evaluate_preds(y_true, y_pred):
  # Make sure float32 dtype (for metric calculation)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various evaluation metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)

  # Reduce to single scalar if with long horizons
  if mae.ndim > 0:
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    mape = tf.reduce_mean(mape)
    mase = tf.reduce_mean(mase)

  metrics = {'MAE': mae.numpy(),
             'MSE': mse.numpy(),
             'RMSE': rmse.numpy(),
             'MAPE': mape.numpy(),
             'MASE': mase.numpy()}
  return metrics

In [ ]:
# Import time series data with Pandas and parse dates
Qassim_Load = pd.read_csv('/content/drive/MyDrive/KSA_Qassim_Energy/NLTP_F_DAILY_Qassim.csv')
Qassim_Load.head()


,Date,Parameter,Parameter_Category,Operating_Area,System_City,System_City_Calculated
0,1-Jan-11,Non-Coincident Peak Load MW,Electrical Grid,EOA,NOA,563.0
1,1-Jan-11,Non-Coincident Peak Load MW,Electrical Grid,EOA,Jubail Industry,2168.0
2,1-Jan-11,Non-Coincident Peak Load MW,Electrical Grid,EOA,HOA,626.0
3,1-Jan-11,Non-Coincident Peak Load MW,Electrical Grid,EOA,DOA,1792.0
4,1-Jan-11,Non-Coincident Peak Load MW,Electrical Grid,NwOA,Tabuk,511.0


In [ ]:
Peak_Load=Qassim_Load[Qassim_Load["System_City"]=='Qassim']

In [ ]:
Peak_Load

,Date,Parameter,Parameter_Category,Operating_Area,System_City,System_City_Calculated
2521,1-May-11,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1125.0
2545,2-May-11,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1186.0
2569,3-May-11,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1275.0
2593,4-May-11,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1321.0
2617,5-May-11,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1255.0
...,...,...,...,...,...,...
126573,8-Oct-21,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1934.0
126594,9-Oct-21,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,1931.0
126615,10-Oct-21,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,2041.0
126636,11-Oct-21,Non-Coincident Peak Load MW,Electrical Grid,COA,Qassim,2052.0


In [ ]:
Peak_Load=Peak_Load[['Date','System_City_Calculated']].rename(columns={'System_City_Calculated': 'Peak_Load'})
Peak_Load.index=pd.DatetimeIndex(Peak_Load['Date'])
Peak_Load.drop('Date',inplace=True,axis=1)

In [ ]:
Peak_Load

,Peak_Load
Date,
2011-05-01,1125.0
2011-05-02,1186.0
2011-05-03,1275.0
2011-05-04,1321.0
2011-05-05,1255.0
...,...
2021-10-08,1934.0
2021-10-09,1931.0
2021-10-10,2041.0


In [ ]:
pop = pd.read_csv('/content/drive/MyDrive/KSA_Qassim_Energy/Qassim.csv')
pop.drop([0],inplace=True)
pop

,year,growthRate,Qassim,Oil_GDP,Non_Oil_GDP,total_GDP
1,2011,0.0383,1251946,1303517,875276,2178792
2,2012,0.0383,1279631,1372719,923978,2296697
3,2013,0.0383,1306957,1370818,987872,2358690
4,2014,0.0383,1334617,1403651,1041190,2444841
5,2015,0.0383,1362085,1467402,1077834,2545236
6,2016,0.0383,1389929,1507207,1080535,2587742
7,2017,0.0383,1423130,1471056,1097537,2568592
8,2018,0.0383,1455693,1567674,1059996,2627670
9,2019,0.0263,1488285,1544708,1095103,2639811
10,2020,0.0251,1525641,1478360,1061139,2539499


In [ ]:
pop=pop.drop(['growthRate'],axis=1)
pop

,year,Qassim,Oil_GDP,Non_Oil_GDP,total_GDP
1,2011,1251946,1303517,875276,2178792
2,2012,1279631,1372719,923978,2296697
3,2013,1306957,1370818,987872,2358690
4,2014,1334617,1403651,1041190,2444841
5,2015,1362085,1467402,1077834,2545236
6,2016,1389929,1507207,1080535,2587742
7,2017,1423130,1471056,1097537,2568592
8,2018,1455693,1567674,1059996,2627670
9,2019,1488285,1544708,1095103,2639811
10,2020,1525641,1478360,1061139,2539499


In [ ]:
temp = pd.read_csv('/content/drive/MyDrive/KSA_Qassim_Energy/temp (4).csv')
temp.index=pd.DatetimeIndex(temp['Date'])

In [ ]:
temp

,Date,Meantemp,CDD
Date,,,
2011-01-01,2011-01-01,15.959153,-6.040847
2011-01-02,2011-01-02,15.969404,-6.030596
2011-01-03,2011-01-03,14.834912,-7.165088
2011-01-04,2011-01-04,14.302114,-7.697886
2011-01-05,2011-01-05,14.505676,-7.494324
...,...,...,...
2032-08-25,2032-08-25,36.722222,14.722222
2032-08-26,2032-08-26,36.554650,14.554650
2032-08-27,2032-08-27,36.138889,14.138889
